<a href="https://colab.research.google.com/github/Shreyash54/HealthMisinfoDetection/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/cleaned_dataset.csv')

# Combine 'statement' and 'description' columns into a single 'text' column
df['text'] = df['statement'] + ' ' + df['description']

In [ ]:
max_words = 10000  # Maximum number of words to keep based on word frequency
max_sequence_length = 200  # Maximum length of input sequences


In [ ]:
# Ensure 'text' column contains string values and convert to lowercase
df['text'] = df['text'].apply(lambda x: str(x).lower())

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=max_sequence_length)


In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['rating'])
y = to_categorical(y)  # One-hot encode the labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
embedding_dim = 100
lstm_units = 64


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(lstm_units))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 64

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/10
7/7 [==============================] - 5s 389ms/step - loss: 1.7617 - accuracy: 0.4450 - val_loss: 1.7168 - val_accuracy: 0.4909
Epoch 2/10
7/7 [==============================] - 2s 236ms/step - loss: 1.5976 - accuracy: 0.5046 - val_loss: 1.4410 - val_accuracy: 0.4909
Epoch 3/10
7/7 [==============================] - 2s 230ms/step - loss: 1.2160 - accuracy: 0.5023 - val_loss: 1.2523 - val_accuracy: 0.4909
Epoch 4/10
7/7 [==============================] - 2s 270ms/step - loss: 1.1844 - accuracy: 0.5183 - val_loss: 1.2185 - val_accuracy: 0.5545
Epoch 5/10
7/7 [==============================] - 3s 456ms/step - loss: 1.1175 - accuracy: 0.5321 - val_loss: 1.1760 - val_accuracy: 0.5000
Epoch 6/10
7/7 [==============================] - 2s 231ms/step - loss: 1.0217 - accuracy: 0.6399 - val_loss: 1.1425 - val_accuracy: 0.5909
Epoch 7/10
7/7 [==============================] - 2s 239ms/step - loss: 0.8757 - accuracy: 0.6812 - val_loss: 1.0926 - val_accuracy: 0.7091
Epoch 8/10
7/7 [====

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

5/5 [==============================] - 0s 28ms/step - loss: 0.8819 - accuracy: 0.7372
Test loss: 0.8819103240966797
Test accuracy: 0.7372262477874756


In [ ]:
# Make predictions on new statements (replace 'new_statements' with your own data)
new_statements = ["Is air pollution linked to greater risk of dementia?"]
new_sequences = tokenizer.texts_to_sequences(new_statements)
new_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_sequences)

1/1 [==============================] - 0s 432ms/step


In [ ]:
decoded_predictions = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
print("Predictions:")
for statement, prediction in zip(new_statements, decoded_predictions):
    print(f'Statement: {statement}')
    print(f'Prediction: {prediction}')

Predictions:
Statement: Is air pollution linked to greater risk of dementia?
Prediction: unproven
